In [29]:
%run DMD_functions.ipynb

**Umjetni primjer**

In [30]:
N=300
m=10
x=np.random.rand(N)
A=np.random.rand(N,N)
A = A/np.linalg.norm(A, 2)

In [31]:
X=np.empty((N,m+1))
X[:,0]=x
for i in range(m):
    X[:,i+1]=A@X[:,i]
X=X+np.random.rand(N,m+1)/N

In [32]:
Z, L = DMD(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)

L

array([ 0.999  +0.j     ,  0.40678+0.57806j,  0.40678-0.57806j,
       -0.1283 +0.76004j, -0.1283 -0.76004j,  0.04609+0.0347j ,
        0.04609-0.0347j , -0.57824+0.j     , -0.52522+0.29946j,
       -0.52522-0.29946j])

In [33]:
#permutirani podaci - druge eigenvalues - kao što bi i očekivali!
print(X[:5,[1,6,8,9]])
temp=X[:,[1,6,8,9]]
X[:,[6,9]] = temp[:,[0,2]]
X[:,[1,8]] = temp[:,[1,3]]
print(X[:5,[1,6,8,9]])

Z_perm, L_perm = DMD(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)

L_perm


[[0.5162  0.52034 0.52102 0.52009]
 [0.55417 0.54722 0.54455 0.54571]
 [0.52545 0.52607 0.52426 0.52569]
 [0.5242  0.5152  0.513   0.51517]
 [0.54164 0.52634 0.52519 0.52271]]
[[0.52034 0.5162  0.52009 0.52102]
 [0.54722 0.55417 0.54571 0.54455]
 [0.52607 0.52545 0.52569 0.52426]
 [0.5152  0.5242  0.51517 0.513  ]
 [0.52634 0.54164 0.52271 0.52519]]


array([ 0.99923+0.j     ,  0.63395+0.58302j,  0.63395-0.58302j,
        0.13754+0.81216j,  0.13754-0.81216j, -0.00076+0.j     ,
       -0.7819 +0.33065j, -0.7819 -0.33065j, -0.42721+0.76027j,
       -0.42721-0.76027j])

In [34]:
Ze, Le = DMD_exact(X[:,:m], X[:, 1:], N*np.finfo(float).resolution)
Le
#isto kao L?

array([ 0.99923+0.j     ,  0.63395+0.58302j,  0.63395-0.58302j,
        0.13754+0.81216j,  0.13754-0.81216j, -0.00076+0.j     ,
       -0.7819 +0.33065j, -0.7819 -0.33065j, -0.42721+0.76027j,
       -0.42721-0.76027j])

In [35]:
Zg, Lg, rg = GEDMD(X[:,:m], X[:,1:], N*np.finfo(float).resolution)
Lg

array([ 0.99923+0.j     ,  0.63395+0.58302j,  0.63395-0.58302j,
        0.13754+0.81216j,  0.13754-0.81216j, -0.00076+0.j     ,
       -0.7819 +0.33065j, -0.7819 -0.33065j, -0.42721+0.76027j,
       -0.42721-0.76027j])

In [36]:
Z_qr, L_qr, r_qr, Q, R = GEDMDQ(X, N*np.finfo(float).resolution)
print(L_qr)

B = (A@X[:,m]+np.random.rand(N)/N).reshape((-1,1))

Z_added, L_added, r_added, _,_ = DMD_added_snap(X, B, Q, R, N*np.finfo(float).resolution)
print(L_added)

#provjera.. - dobije se isto
_,L_added_direct,_,_,_ = GEDMDQ(np.concatenate((X, B), axis=1), N*np.finfo(float).resolution)
print(L_added_direct)

print(np.allclose(L_added, L_added_direct))


[ 0.99923+0.j       0.63395+0.58302j  0.63395-0.58302j  0.13754+0.81216j
  0.13754-0.81216j -0.00076+0.j      -0.7819 +0.33065j -0.7819 -0.33065j
 -0.42721+0.76027j -0.42721-0.76027j]
[ 0.99968+0.j       0.64009+0.54578j  0.64009-0.54578j  0.22311+0.73053j
  0.22311-0.73053j  0.00139+0.j      -0.28111+0.77166j -0.28111-0.77166j
 -0.84154+0.j      -0.60566+0.46996j -0.60566-0.46996j]
[ 0.99968+0.j       0.64009+0.54578j  0.64009-0.54578j  0.22311+0.73053j
  0.22311-0.73053j  0.00139+0.j      -0.28111+0.77166j -0.28111-0.77166j
 -0.84154+0.j      -0.60566+0.46996j -0.60566-0.46996j]
True


In [37]:
Z_qr, L_qr, r_qr, Q, R = GEDMDQ(X, N*np.finfo(float).resolution)
print(L_qr)

l=2

Z_disc, L_disc, r_disc, _,_,_ = DMD_discarding_snap(l, Q, R, N*np.finfo(float).resolution)
print(L_added)

#provjera.. - dobije se isto
_,L_discarded_direct,_,_,_ = GEDMDQ(X[:,l:], N*np.finfo(float).resolution)
print(L_added_direct)

print(np.allclose(L_added, L_added_direct))

[ 0.99923+0.j       0.63395+0.58302j  0.63395-0.58302j  0.13754+0.81216j
  0.13754-0.81216j -0.00076+0.j      -0.7819 +0.33065j -0.7819 -0.33065j
 -0.42721+0.76027j -0.42721-0.76027j]
[ 0.99968+0.j       0.64009+0.54578j  0.64009-0.54578j  0.22311+0.73053j
  0.22311-0.73053j  0.00139+0.j      -0.28111+0.77166j -0.28111-0.77166j
 -0.84154+0.j      -0.60566+0.46996j -0.60566-0.46996j]
[ 0.99968+0.j       0.64009+0.54578j  0.64009-0.54578j  0.22311+0.73053j
  0.22311-0.73053j  0.00139+0.j      -0.28111+0.77166j -0.28111-0.77166j
 -0.84154+0.j      -0.60566+0.46996j -0.60566-0.46996j]
True


In [38]:
np.linalg.eigvals(A)[:10]

array([0.99898+0.j     , 0.02836+0.01701j, 0.02836-0.01701j,
       0.03059+0.01039j, 0.03059-0.01039j, 0.03194+0.00423j,
       0.03194-0.00423j, 0.03208+0.j     , 0.0235 +0.02202j,
       0.0235 -0.02202j])

In [39]:
#!pip install pydmd

In [40]:
import pydmd
dmd1 = pydmd.DMD(svd_rank=10)
dmd1.fit(X[:,:m], X[:,1:])
dmd1.eigs

array([ 0.99923+0.j     ,  0.63395+0.58302j,  0.63395-0.58302j,
        0.13754+0.81216j,  0.13754-0.81216j, -0.00076+0.j     ,
       -0.7819 +0.33065j, -0.7819 -0.33065j, -0.42721+0.76027j,
       -0.42721-0.76027j])

In [41]:
for i in range(dmd1.eigs.shape[0]):
    print(np.linalg.norm(dmd1.reconstructed_data[:,i]-X[:,i]))

5.1154936870127375e-15
7.21764445522285e-15
1.5971483166014223e-14
2.8259202075615845e-14
4.4831933905436203e-14
6.178416485860473e-14
7.868587321042017e-14
9.046321783015468e-14
1.0377896479861422e-13
1.2051301814157545e-13


In [42]:
dmd1.reconstructed_data.shape

(300, 10)

In [43]:
#usporedi s rezultatima lin regresije...

In [44]:
#procjena s_i - ne ovako!
for i in range(m):
    s=np.zeros((Z.shape[0]))
    for j in range(Z.shape[1]):
        s=s+Z[:,j]*(L[j]**(i))
    print("norma razlike:",np.linalg.norm(s-X[:,i]))
    #print(s,X[:,i], sep="\n")

norma razlike: 9.021169168641855
norma razlike: 8.08759588230142
norma razlike: 8.088077094008236
norma razlike: 8.02174493994071
norma razlike: 8.173084479393864
norma razlike: 8.035732672781334
norma razlike: 8.059441781751463
norma razlike: 7.980146462647608
norma razlike: 7.991105555337669
norma razlike: 7.986652538919365


*proba za funkcije...*

In [45]:
#QR za (X Y) kad X i Y imaju "istu jezgru"
N=20; m=7
A = np.random.rand(N,N)
A=A/np.linalg.norm(A)
x = np.ones((N))
X=np.empty((N,m+1))
X[:,0] = x
for i in range(m):
    X[:,i+1]= A @ X[:,i]
#print(X)

Z = np.concatenate((X[:,:m], X[:,1:]), axis=1)
print(Z.shape)

Q, R = np.linalg.qr(Z, mode="reduced")

R_12 = R[:m,m:]
R_22 = R[m:2*m, m:]

Q_1 = Q[:,:m]
Q_2 = Q[:,m:]


print(np.allclose(X[:,1:],Q_1@R_12 + Q_2@R_22))
print(np.allclose(X[:,1:],Q_1@R_12 + Q_2*R_22[:,m-1]))
print(R_22)
print(R.shape)

(20, 14)
True
True
[[ 0. -0. -0.  0. -0.  0. -0.]
 [ 0.  0. -0.  0.  0.  0. -0.]
 [ 0.  0. -0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -0. -0.]
 [ 0.  0.  0.  0.  0. -0.  0.]
 [ 0.  0.  0.  0.  0.  0. -0.]]
(14, 14)


In [46]:
"""HH transformacija - proba s kvadratnom matricom koja skoro gornje trokutasta (dvije sporedne diag ne-nula)"""
dim_trapez_m = 6

np.set_printoptions(precision=8, suppress=True)

trapez_m = np.random.rand(dim_trapez_m,dim_trapez_m)
trapez_m = np.triu(trapez_m, -2)
print(trapez_m)
trapez_copy = trapez_m.copy()
v_all=np.empty((3,dim_trapez_m-1))
#hh1 = np.eye(dim_trapez_m, dim_trapez_m)


#HH transf od R (trapezoidalna) 
#### racun od HH matrice ("Q") bi se mogao i ovdje dogoditi - primjena transformacija na I i onda uzmemo transponirano
for i in range(dim_trapez_m-2):
    element=trapez_m[i:(i+3),i]  
    v = element.copy().reshape(-1,1) #ako stavim v=element - pokazuju na istu memoriju; ne radi!!
    v[0] = v[0] + np.sign(element[0])*np.linalg.norm(element)
    trapez_m[i:(i+3),i:] = trapez_m[i:(i+3),i:] - 2/(v.T@v)*v@(v.T@trapez_m[i:(i+3),i:])
    #hh1[i:(i+3),i:] = hh1[i:(i+3),i:] - 2/(v.T@v)*v@(v.T@hh1[i:(i+3),i:])  #- daje krivo iz nekog razloga..
    #print("hh1",hh1,sep="\n")
    v_all[:,i] = v.reshape(-1)

#predzadnji stupac
element=trapez_m[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2]  
v = element.copy().reshape(-1,1)
v[0] = v[0] + np.sign(element[0])*np.linalg.norm(element)
trapez_m[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:] = trapez_m[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:] - 2/(v.T@v)*v@(v.T@trapez_m[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:])
#hh1[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:] = hh1[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:] - 2/(v.T@v)*v@(v.T@hh1[dim_trapez_m-2:dim_trapez_m,dim_trapez_m-2:])
v_all[:3,dim_trapez_m-2] = np.concatenate((v.reshape(-1),[0]), axis=0)

#print("hh1_fin",hh1.T, sep="\n")

##Q - from HH matrices
hh_current = np.eye(dim_trapez_m, dim_trapez_m)
v_current=v_all[:2,-1].reshape(-1,1)
hh_current[-2:,-2:] = np.eye(2,2)-2/(v_current.T@v_current)*(v_current@v_current.T)

hh=hh_current #ovdje @ I
#print(hh)

#print(v_all)
for i in range(dim_trapez_m-3,-1,-1):
    v_current = v_all[:,i].reshape(-1,1)
    hh_current = np.eye(3,3)-2/(v_current.T@v_current)*(v_current@v_current.T)
    hh[i:i+3, :] = hh_current@hh[i:i+3, :]
    #print(hh)
print("hh_pravi",hh, sep="\n")

print(trapez_m)
print(np.allclose(hh@trapez_m,trapez_copy))
np.set_printoptions()

[[0.98920538 0.80767726 0.04154417 0.86364134 0.11265656 0.31472834]
 [0.9644896  0.73224148 0.83110231 0.64886022 0.26514759 0.56896933]
 [0.77521436 0.03773853 0.594696   0.46226886 0.43391382 0.2294295 ]
 [0.         0.11362446 0.91260447 0.4928366  0.64514135 0.21668058]
 [0.         0.         0.53997735 0.74815222 0.25257528 0.12050616]
 [0.         0.         0.         0.04276111 0.83355473 0.97054186]]
hh_pravi
[[-0.62441513 -0.39433161  0.36880592 -0.44019306  0.07416617 -0.34542723]
 [-0.60881382 -0.27707733 -0.28312101  0.47924846 -0.11244067  0.47968155]
 [-0.48933779  0.84791148 -0.11836407 -0.03455665  0.04525494 -0.15602059]
 [ 0.         -0.22085484 -0.75772699  0.05203495  0.18238621 -0.58403817]
 [ 0.          0.         -0.44233974 -0.7536495  -0.19073183  0.44717932]
 [ 0.          0.          0.         -0.06827604  0.95402574  0.29184461]]
[[-1.58421109 -0.96859152 -0.82293461 -1.16051141 -0.44410041 -0.65518605]
 [-0.         -0.51447577  0.05603466 -0.23722782 

In [47]:
#test hh fje
N0=7; N=2
l=N0-N
trapez_m = np.random.rand(N0,N)
trapez_m = np.triu(trapez_m, -l)
trapez_copy=trapez_m.copy()
np.set_printoptions(precision=5, suppress=True)
print(trapez_m)

hh, R,_ = householder_for_dmd(trapez_m, l)
print(R)
print(np.allclose(hh@R, trapez_copy))

np.set_printoptions()

[[0.43095 0.2774 ]
 [0.36064 0.86425]
 [0.5942  0.41578]
 [0.91904 0.71336]
 [0.1143  0.15826]
 [0.03327 0.31017]
 [0.      0.21323]]
[[-1.23599 -1.1022 ]
 [ 0.      -0.67639]
 [ 0.      -0.     ]
 [ 0.      -0.     ]
 [ 0.       0.     ]
 [ 0.       0.     ]
 [ 0.       0.     ]]
True
